In [1]:
import sqlite3

conn = sqlite3.connect('my_db.sqlite3')
curs = conn.cursor()
curs.execute("CREATE TABLE new_table (element TEXT)")
conn.close()

In [2]:
conn = sqlite3.connect('my_db.sqlite3')
curs = conn.cursor()
curs.execute("INSERT INTO new_table VALUES ('text2')")
conn.commit()
conn.close()

In [3]:
conn = sqlite3.connect('my_db.sqlite3')
curs = conn.cursor()
curs.execute("SELECT * FROM new_table")

print(curs.fetchall())

conn.close()

[('text2',)]


1. Інформація про товари
2. Прихід, розхід товару
3. Залишки товарів
4. Таблиця із прайсами
5. Метрики: залишки, оборотність, XYZ-аналіз, к-сть днів зберігання ... 
            динаміку розходу, маржу
            візуалізувати: power bi + py

In [47]:
import sqlite3

def create_db():
    conn = sqlite3.connect('my_db.sqlite3')
    curs = conn.cursor()

    script = """CREATE TABLE product ( 
                    code TEXT PRIMARY KEY,
                    product_name TEXT NOT NULL,
                    COST REAL NOT NULL
                    );

    CREATE TABLE price (
                    product_code TEXT,
                    price_start_date DATETIME NOT NULL,
                    price REAL NOT NULL,

                    FOREIGN KEY (product_code) REFERENCES product (code)
                    );

    CREATE TABLE stock (
                    id INTEGER PRIMARY KEY AUTOINCREMENT, 
                    product_code TEXT,
                    period DATETIME NOT NULL,
                    type TEXT NOT NULL, 
                    qty REAL NOT NULL,

                    FOREIGN KEY (product_code) REFERENCES product (code)
                    )
    """

    curs.executescript(script)
    conn.close()
    
def parse_file(file_name):
    list_with_product = []
    first_row = True
    
    for string in open(file_name, encoding='UTF-8'):
        if first_row == False:
            tmp = string.split('|')

            code = tmp[0]
            product_name = tmp[1:-1][0]
            cost = float(tmp[-1].rstrip(' грн\n'))
            list_with_product.append( (code, product_name, cost) ) 

        else:
            first_row = False
    return list_with_product

def add_products():
    list_with_product = parse_file('file.txt')
    
    for product_info in list_with_product:
        conn = sqlite3.connect('my_db.sqlite3')
        curs = conn.cursor()

        s = "INSERT INTO product VALUES (?, ?, ?)"
        try:
            curs.execute(s, product_info)
            conn.commit()
        except Exception as e:
            print(e)

        conn.close()
        
        

    
    
def qty_on_stock(product_code=None):
    conn = sqlite3.connect('my_db.sqlite3')
    curs = conn.cursor()
    if product_code == None:
        s = """SELECT product.code, product.product_name, SUM(stock.qty) 
                FROM stock
                    JOIN product ON product.code = stock.product_code
                GROUP BY product.code, product.product_name
                ORDER BY SUM(stock.qty) DESC
                
            """
        try:
            curs.execute(s)
            for el in curs.fetchall():
                print(el)
        except Exception as e:
            print(e)
        
    else:
        s = """SELECT product.code, product.product_name, SUM(stock.qty) 
                FROM stock
                    JOIN product ON product.code = stock.product_code
                WHERE product.code = (?)
                GROUP BY product.code, product.product_name
                ORDER BY SUM(stock.qty) DESC
                
            """
        try:
            curs.execute(s, (product_code, ) )
            
            return curs.fetchall()[0][-1]
            
        except Exception as e:
            print(e)
        
    
    conn.close()
    

def move_on_stock():
    
    from datetime import datetime 
    
    type_move = input('1-Прихід \ 2-Розхід ')
    product = input('Введіть артикул товару ')
    qty = float(input('Введіть к-сть товару для руху '))
    
    
    conn = sqlite3.connect('my_db.sqlite3')
    curs = conn.cursor()
    
    if type_move == '2' :
        qty_product = qty_on_stock(product)
       
        print(qty_product)
        if qty_product < qty:
            print('Такої кількості немає в наявності')
            return None
        else:
            qty = -qty
            s = "INSERT INTO stock (product_code, period, type, qty) VALUES (?, ?, ?, ?)"
    if type_move == '1':
        s = "INSERT INTO stock (product_code, period, type, qty) VALUES (?, ?, ?, ?)"
    
    try:
        curs.execute(s, (product, datetime.now(), type_move, qty) ) 
        conn.commit()
        print('OK')
    except Exception as e:
        print(e)
    conn.close()
    
    
def add_price(date=None):
    from datetime import datetime
    from random import randrange
    
    if date == None:
        date = datetime.now()
        
    s = "SELECT code, COST FROM product"
    l = "INSERT INTO price (product_code, price_start_date, price) VALUES (?, ?, ?)" 
    conn = sqlite3.connect('my_db.sqlite3')
    curs = conn.cursor()
    
    for el in curs.execute(s).fetchall():
        product = el[0]
        cost = el[1] * (1 + randrange(10, 20)/100 )
        
        curs.execute(l, (product, date, cost))
        conn.commit()
        
    
    
    conn.close()
                

    
        

In [50]:
# main
qty_on_stock('B81410')
    
    
    

280.0